## Dogs vs Cats

In [2]:
import pandas as pd

In [3]:
PATH = "d:\\data\\fastai_data\\dogscats\\"
sz=224
batch_size=64

In [4]:
import os
import skimage
import numpy as np
from skimage import io
from scipy import misc
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input

C:\Users\anshu\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
train_data_dir = f'{PATH}train'
validation_data_dir = f'{PATH}valid'

In [23]:
join = os.path.join
kaggle_dogscats_path = "d:\\data\\kaggle_data\\dogs_cats\\test1"
def get_test_data(path):
    data = []
    ids = []
    all_files_names = os.listdir(path)
    all_files_in_path = [join(path, name) for name in os.listdir(path)]
    all_files_names = [item.split(".")[0] for item in all_files_names]
    for file_name, file_path in zip(all_files_names, all_files_in_path):
        image = io.imread(file_path)
        #image = skimage.transform.resize(image, (sz,sz))
        image = misc.imresize(image, (sz,sz))
        yield preprocess_input(image), file_name
        #data.append(image)
        #ids.append(file_name)
        
    """data = np.array(data)/255.0
    ids = np.array(ids)
    print(data.shape, ids.shape)
    return data, ids"""
        
#test_data, test_ids = get_test_data(kaggle_dogscats_path)        
        
    
    
    

In [7]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    shuffle=False,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

test_generator = test_datagen.flow_from_directory(kaggle_dogscats_path, shuffle=False, target_size=(sz, sz), batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [8]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

C:\Users\anshu\Anaconda3\lib\site-packages\keras_applications\resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [9]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=3, workers=4,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/3
359/359 [==============================] - 237s 660ms/step - loss: 0.2002 - acc: 0.9496 - val_loss: 0.0447 - val_acc: 0.9875
Epoch 2/3
359/359 [==============================] - 141s 393ms/step - loss: 0.0868 - acc: 0.9699 - val_loss: 0.0609 - val_acc: 0.9845
Epoch 3/3
359/359 [==============================] - 142s 395ms/step - loss: 0.0711 - acc: 0.9748 - val_loss: 0.1618 - val_acc: 0.9700
Wall time: 8min 39s


In [24]:
with open('sub_1.csv', 'w') as fp:
    fp.write("id,label\n")
    for image, id in get_test_data(kaggle_dogscats_path):
        pred = model.predict(np.array([image]))[0][0]
        pred = 1 if pred > 0.5 else 0
        fp.write("{},{}\n".format(id, pred))
        
        
    

C:\Users\anshu\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':
